In [1]:
import pandas as pd
import duckdb

176. Вторая по величине зарплата  
Напишите решение для поиска второй по величине уникальной заработной платы в Employeeтаблице. Если второй по величине заработной платы нет, верните значение  null (return None in Pandas).

In [2]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,100
1,2,200
2,3,300


In [17]:
data = [[1, 0], [2, 0], [3, 1]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,0
1,2,0
2,3,1


In [ ]:
# мое решение
employee = employee.drop_duplicates(subset=['salary']) 
employee['rank'] = employee['salary'].rank(method='dense', ascending=False).astype('int64')
data_1 = [1,2]
ranke_1 = pd.DataFrame(data_1, columns=['ranke_1']).astype({'ranke_1':'int64'})
union_table = ranke_1.merge(employee, left_on='ranke_1', right_on='rank', how='left')
union_table[union_table['ranke_1'] == 2][['salary']].rename(columns={'salary': 'SecondHighestSalary'})

In [ ]:
# решение GPT
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
result = (
    salaries.iloc[1:2]
    .to_frame(name='SecondHighestSalary')
)
if result.empty:
    result = pd.DataFrame({'SecondHighestSalary': [None]})

,SecondHighestSalary
0,0


In [19]:
query = """
with t1 as (
    select 
        salary,
        dense_rank() over(order by salary desc) as rank
    from employee
    )
select
    case when count(*) < 1 then null
         else max(salary) end as SecondHighestSalary
from t1
where rank > 1
"""
result = duckdb.query(query).to_df()
result

,SecondHighestSalary
0,0


177. N-я самая высокая зарплата  
Напишите решение для поиска самой высокой уникальной заработной платы в таблице. Если уникальных зарплат  меньше, чем указано , верните значение .nthEmployeennull

In [8]:
N = 2
col_name = f'getNthHighestSalary({N})'
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
if N > 0 and len(salaries) >= N:
    result = salaries.iloc[N-1:N].to_frame(name=col_name)
else:
    result = pd.DataFrame({col_name: [None]})
result

,getNthHighestSalary(2)
1,200


In [ ]:
N = 2
query = """
with t1 as (
    select 
        e.salary,
        dense_rank() over(order by e.salary desc) as rank
    from employee e
    )
select
    case when count(*) < 1 or N <= 0 then null
        else max(t1.salary) end as SecondHighestSalary
from t1
where rank > N - 1
"""
result = duckdb.query(query).to_df()
result